In [1]:
import pandas as pd
import glob

In [2]:
# The idea is load and treat 60 days of the big data Italia to use as dataset training and test.

folder_path = '/home/nilton/Arquivos/Datasets/dataverse_files'
file_list = glob.glob(folder_path + "/*.txt")

In [3]:
file_list

['/home/nilton/Arquivos/Datasets/dataverse_files/sms-call-internet-mi-2013-11-14.txt',
 '/home/nilton/Arquivos/Datasets/dataverse_files/sms-call-internet-mi-2013-12-04.txt',
 '/home/nilton/Arquivos/Datasets/dataverse_files/sms-call-internet-mi-2013-12-27.txt',
 '/home/nilton/Arquivos/Datasets/dataverse_files/sms-call-internet-mi-2013-12-21.txt',
 '/home/nilton/Arquivos/Datasets/dataverse_files/sms-call-internet-mi-2013-11-09.txt',
 '/home/nilton/Arquivos/Datasets/dataverse_files/sms-call-internet-mi-2013-12-08.txt',
 '/home/nilton/Arquivos/Datasets/dataverse_files/sms-call-internet-mi-2013-11-15.txt',
 '/home/nilton/Arquivos/Datasets/dataverse_files/sms-call-internet-mi-2013-11-02.txt',
 '/home/nilton/Arquivos/Datasets/dataverse_files/sms-call-internet-mi-2013-12-25.txt',
 '/home/nilton/Arquivos/Datasets/dataverse_files/sms-call-internet-mi-2013-12-20.txt',
 '/home/nilton/Arquivos/Datasets/dataverse_files/sms-call-internet-mi-2013-11-19.txt',
 '/home/nilton/Arquivos/Datasets/dataverse_

In [4]:
def convert_time(dataset2):
    dataset = dataset2.copy()
    minuto = 0.0
    hora = 0.0
    horario = 0.0
    anterior = dataset.iloc[0,0]
    for i in range(0,dataset.shape[0]):
        if dataset.iloc[i,0] == anterior:
            horario = minuto + hora
            dataset.iat[i,1] = horario
            minuto += 0.10
            if minuto == 0.60:
                minuto = 0.0
                hora += 1.0
        else:
            horario = 0.0
            minuto = 0.0
            hora = 0.0
            dataset.iat[i,1] = horario
            minuto += 0.10
        anterior = dataset.iloc[i,0]
    return dataset

In [5]:
# Other lambda functions used to transform fronthaul_traffic
fronthaul_band = lambda x: x*(100/20)*(8/6)*(8/2) # X in Mbps
Gbps_Mbps = lambda x: x*1000
Mbps_Gbps = lambda x: x/1000

In [11]:
# We need to apply every change we did in order to save memory and cpu before concatenate the datasets.

def applying_changes(dataset2):
    dataset = dataset2.copy()
    dataset.columns = ['Square_id', 'Time_Interval', 'Country_code', 'SMS_in', 
                'SMS_out', 'Call-in', 'Call-out', 'Internet']
    squares = [5161, 5059, 5758, 5955, 5061, 5262, 4857, 5162, 4755, 5458, 4654, 4856]
    dataset = dataset[dataset['Square_id'].isin(squares)]
    dataset = dataset.drop(['SMS_in', 'SMS_out', 'Call-in', 'Call-out'], axis = 1)
    dataset = dataset[dataset['Country_code'] == 39]
    dataset['Internet'].interpolate(method ='linear')
    dataset['Traffic_MB'] = dataset['Internet']*5000
    dataset['Traffic_Gbps'] = dataset['Traffic_MB']/(125*600)
    dataset['Traffic_Mbps'] = list(map(Gbps_Mbps,dataset['Traffic_Gbps']))
    dataset['Traffic_fronthaul'] = list(map(fronthaul_band,dataset['Traffic_Mbps']))
    dataset['Fronthaul_Gbps'] = list(map(Mbps_Gbps,dataset['Traffic_fronthaul']))
    dataset = convert_time(dataset)
    dataset = dataset.groupby(['Time_Interval'])['Fronthaul_Gbps'].sum().reset_index()
    return dataset

In [16]:
main_dataframe = pd.DataFrame(pd.read_csv(file_list[30],sep = '\t', header = None))
main_dataframe = applying_changes(main_dataframe)


In [17]:
main_dataframe.head()

,Time_Interval,Fronthaul_Gbps
0,0.0,10815.173381
1,0.1,10742.386063
2,0.2,10610.969802
3,0.3,12007.490774
4,0.4,9702.333183


In [18]:
# Here we load, treat, and concatenate the datasets
for i in range(30,len(file_list)):
    data = pd.read_csv(file_list[i],sep = '\t', header = None)
    data = applying_changes(data)
    main_dataframe = pd.concat([main_dataframe,data],axis=0)

In [19]:
# This is teh second part, however, the same code must be used to produce part 1. The range would be (1,30)
main_dataframe.to_csv('main_pt2.csv')